<a href="https://colab.research.google.com/github/Brent-Morrison/Coursera_Capstone/blob/master/week_3_clustering_toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Segmenting and Clustering Neighborhoods in Toronto
The section of code below satisfies the first requirement of the project, that described under points 1 through 4

In [1]:
# Required libraries
import numpy as np
import pandas as pd
import requests
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import time

### Grab data from wikipedia
Load the Canadian postal codes beginning with M.  Postal codes beginning with M are located in Toronto

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_codes_list = pd.read_html(url)
toronto_codes_list

[    Postal Code  ...                                       Neighborhood
 0           M1A  ...                                       Not assigned
 1           M2A  ...                                       Not assigned
 2           M3A  ...                                          Parkwoods
 3           M4A  ...                                   Victoria Village
 4           M5A  ...                          Regent Park, Harbourfront
 ..          ...  ...                                                ...
 175         M5Z  ...                                       Not assigned
 176         M6Z  ...                                       Not assigned
 177         M7Z  ...                                       Not assigned
 178         M8Z  ...  Mimico NW, The Queensway West, South of Bloor,...
 179         M9Z  ...                                       Not assigned
 
 [180 rows x 3 columns],
                                                   0   ...   17
 0                               

The pandas 'read_html' function returns a list of dataframes.
We now need to extract the table of interest.

In [3]:
toronto_codes = toronto_codes_list[0]
toronto_codes

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


### Clean data
The requirements of the assignment include a number of steps to transform  the data into an appropriate format for further analysis.

Ignore cells with a borough that is 'Not assigned'.

In [4]:
# Use copy to ensure 'SettingWithCopyWarning' is not triggered downstream
toronto_codes = toronto_codes[toronto_codes['Borough'] != 'Not assigned'].copy()
toronto_codes.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
len(toronto_codes)

103

We have removed 77 rows.

Next we are asked to remove duplicates when more than one Neighbourhood exists for a Postal Code.  

The example given is for M5A which we expect to be listed twice for the Neighbourhoods 'Regent Park' and 'Harbourfront'.
Inspection of the dataframe above reveals that the duplicate has already been accounted for with these two Neighbourhoods concatenated into the single entry for Postal code M5A.

Lets see if there are any duplicate Postal Codes.  If so we will need to implement the transform required by the assignment.  If not we will proceed.

In [6]:
toronto_codes[toronto_codes.duplicated(subset=['Postal Code'])]

,Postal Code,Borough,Neighborhood


There are no duplicate Postal Codes.  It looks like the Wikipedia list has been updated since this project was designed.  On that basis we will move to the next task.

Next we are asked to alter the Neighbourhood column.  If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
toronto_codes['Neighborhood'] = np.where(toronto_codes['Neighborhood'] == 'Not assigned', toronto_codes['Borough'], toronto_codes['Neighborhood'])

In [8]:
toronto_codes

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [9]:
toronto_codes.shape

(103, 3)

***This concludes the section of the assignment covered by the first link.***

## Latitude and Longitude data

This section requires grabbing latitude and logitude data for Toronto via either the geocoder library, or in the case this does not work, via a csv provided.

Firstly install geocoder.

In [10]:
!pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 2.3MB/s 


The code below tests the geocoder library, modifying the code provided in the assignment.  Modification is required to prevent an infinite loop should the geocoder library not work.

In [11]:
postal_code = 'M3A'
attempt = 0
attempt_limit = 25

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None and attempt < attempt_limit):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng
  attempt += 1

if lat_lng_coords is not None:
  latitude = lat_lng_coords[0]
  longitude = lat_lng_coords[1]
  print(latitude)
  print(longitude)
else:
  print('goecoder unable to return result after {} attempts.'.format(attempt_limit))

goecoder unable to return result after 25 attempts.


True to form, the geocoder library has proven unreliable.  We will use the csv file provided.

In [12]:
url = 'http://cocl.us/Geospatial_data'
toronto_latlong = pd.read_csv(url)
toronto_latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
toronto_latlong.shape

(103, 3)

In [14]:
# ERROR RESOLUTION NOTES
# Row 4 doesn't work, throwing a 'ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat' error
# How to perform a left join when a dataframe has differing types?  Must be 'merge'.
# toronto_all = toronto_codes.join(toronto_latlong, on = 'Postal Code', how='left')
toronto_all = toronto_codes.merge(toronto_latlong, on = 'Postal Code', how='left')
toronto_all.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [15]:
toronto_all.shape

(103, 5)

Determine the unique Boroughs.

In [16]:
toronto_all['Borough'].unique()

# Alternate approach
#toronto_all.Borough.unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

## Use the Geocoder package or the csv file to create a dataframe.


***This cell supports the requirements for the second link.***

Note that I have returned only Toronto Boroughs using the string matching functionality of `str.contains`

In [17]:
toronto_boroughs = toronto_all[toronto_all['Borough'].str.contains('Toronto')].copy()
toronto_boroughs.head(15)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [18]:
toronto_boroughs.shape

(39, 5)

## Explore and cluster the neighbourhoods in Toronto

We will cluster neighbourhoods based on the type of shops present using a similar function to that employed for the New York data.  

First the Foursquare credentials.

In [19]:
CLIENT_ID='0M35HSSAZ1L2QPGP5LAKSPMBUUDTERESBKXLNZRQMUCEVJYS'
CLIENT_SECRET='QWIGICNR1KXB3RU5Y5LRJFH03RANUYOVRKNTUTO0KCONHAGQ'
VERSION='20200623'
LIMIT=100

Using the function from the course exercise to return venue information.

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        #results = requests.get(url).json()["response"]['groups'][0]['items']
        # Alternate approach due to KeyError on "groups"
        results = requests.get(url).json()["response"].get('groups')[0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        # Prevent disconnect
        time.sleep(0.05)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Calling the function to create a dataframe.

In [21]:
toronto_venues = getNearbyVenues(names=toronto_boroughs['Neighborhood'],
                                latitudes=toronto_boroughs['Latitude'],
                                longitudes=toronto_boroughs['Longitude']
                                )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

View the dataframe.

In [22]:
toronto_venues.head(20)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park
5,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
6,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
7,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
8,"Regent Park, Harbourfront",43.65426,-79.360636,The Extension Room,43.653313,-79.359725,Gym / Fitness Center
9,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


### Pre-processing the data for ingestion to the clustering algorithm

Calculate the percentage of each type of venue in each neighbourhood.

In [23]:
toronto_venues['venues_per_hood_ctgry'] = toronto_venues.groupby(['Neighborhood','Venue Category'])['Venue Category'].transform('count')
toronto_venues['venues_per_hood'] = toronto_venues.groupby(['Neighborhood'])['Neighborhood'].transform('count')
toronto_venues['venue_perc'] = toronto_venues['venues_per_hood_ctgry']/toronto_venues['venues_per_hood']

Inspect the resultant dataframe.

In [24]:
#toronto_venues[toronto_venues['Venue Category'] == 'Bakery']
toronto_venues[toronto_venues['Neighborhood'] == 'Garden District, Ryerson'].sort_values('venue_perc', ascending=False).head(20)  #Garden District, Ryerson / Dufferin, Dovercourt Village

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,venues_per_hood_ctgry,venues_per_hood,venue_perc
75,"Garden District, Ryerson",43.657162,-79.378937,UNIQLO ユニクロ,43.655910,-79.380641,Clothing Store,9,100,0.09
168,"Garden District, Ryerson",43.657162,-79.378937,Urban Outfitters,43.654411,-79.380055,Clothing Store,9,100,0.09
139,"Garden District, Ryerson",43.657162,-79.378937,Magic Tailor,43.653742,-79.379745,Clothing Store,9,100,0.09
147,"Garden District, Ryerson",43.657162,-79.378937,Hollister Co.,43.654480,-79.380914,Clothing Store,9,100,0.09
131,"Garden District, Ryerson",43.657162,-79.378937,Roots,43.653613,-79.380244,Clothing Store,9,100,0.09
171,"Garden District, Ryerson",43.657162,-79.378937,Ted Baker,43.652843,-79.380325,Clothing Store,9,100,0.09
158,"Garden District, Ryerson",43.657162,-79.378937,Abercrombie & Fitch,43.652915,-79.380495,Clothing Store,9,100,0.09
96,"Garden District, Ryerson",43.657162,-79.378937,Nordstrom,43.655041,-79.380966,Clothing Store,9,100,0.09
164,"Garden District, Ryerson",43.657162,-79.378937,American Eagle Store,43.653293,-79.380088,Clothing Store,9,100,0.09
121,"Garden District, Ryerson",43.657162,-79.378937,Second Cup,43.656027,-79.380575,Coffee Shop,8,100,0.08


Summarise further.  

For each neighbourhood and venue category combination we list the percentage of total venues for that neighbourhood.

In [25]:
# Using 'max()' to return the non nil percentage value
toronto_smry = toronto_venues.groupby(['Neighborhood','Venue Category'])['venue_perc'].max().copy().reset_index()
toronto_smry[toronto_smry['Neighborhood'] == 'Garden District, Ryerson'].sort_values('venue_perc', ascending=False)

,Neighborhood,Venue Category,venue_perc
360,"Garden District, Ryerson",Clothing Store,0.09
361,"Garden District, Ryerson",Coffee Shop,0.08
376,"Garden District, Ryerson",Hotel,0.03
364,"Garden District, Ryerson",Cosmetics Shop,0.03
379,"Garden District, Ryerson",Japanese Restaurant,0.03
...,...,...,...
351,"Garden District, Ryerson",Bakery,0.01
383,"Garden District, Ryerson",Mexican Restaurant,0.01
385,"Garden District, Ryerson",Miscellaneous Shop,0.01
386,"Garden District, Ryerson",Modern European Restaurant,0.01


Finally we reshape this data frame from a long to wide format.

In [26]:
toronto_pvt = toronto_smry.pivot_table(index='Neighborhood', columns='Venue Category', values='venue_perc', fill_value=0).copy()
toronto_pvt

Venue Category,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,...,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.017544,0.035088,0.000000,0.000000,0.00,0.017544,0.017544,0.000000,0.035088,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.035088,0.000000,0.017544,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000,0.017544,0.000000,0.000000,0.017544,0.00,0.00,0.017544,0.000000,0.00,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.00000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.041667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.041667,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.041667,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.066667,0.00,...,0.000000,0.000000,0.000000,0.000000,0.066667,0.00,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.0000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.058824,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.

## Cluster 

We are now ready to apply the clustering algorithm.

In [27]:
# Number of clusters
kclusters = 8

toronto_boroughs_clustering = toronto_pvt.drop(['Neighborhood'], axis = 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_boroughs_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50] 

array([6, 0, 2, 6, 6, 0, 6, 6, 2, 6, 2, 6, 1, 6, 6, 2, 2, 2, 5, 2, 4, 6,
       2, 6, 6, 6, 4, 3, 2, 6, 2, 6, 2, 2, 2, 7, 2, 6, 2], dtype=int32)

In [28]:
# add clustering labels
toronto_boroughs.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_boroughs.head(15)

,Cluster Labels,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,6,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,6,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,6,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,2,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,6,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


Visualise clusters

In [29]:
# Toronto co-ordinates lifted from the dataframe above. 
latitude = 43.65
longitude = -79.4

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_boroughs['Latitude'], toronto_boroughs['Longitude'], toronto_boroughs['Neighborhood'], toronto_boroughs['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine clusters

In [31]:
# Join the 'borough' data frame that contains cluster labels and the 'toronto_smry' frame containg the percentage of types of venue for each neighbourhood
toronto_inspect = toronto_boroughs.merge(toronto_smry, on = 'Neighborhood', how='left')

# Sum the percentage of type of venue for each cluster
toronto_inspect = toronto_inspect.groupby(['Cluster Labels', 'Venue Category'])['venue_perc'].sum().reset_index()

# Sort and inspect
toronto_inspect = toronto_inspect.sort_values(['Cluster Labels','venue_perc'], ascending=[True, False])
toronto_inspect

,Cluster Labels,Venue Category,venue_perc
16,0,Coffee Shop,0.387719
22,0,Diner,0.084211
9,0,Beer Bar,0.068421
13,0,Café,0.068421
15,0,Cocktail Bar,0.052632
...,...,...,...
515,7,Sandwich Place,0.021277
516,7,Snack Place,0.021277
517,7,Sushi Restaurant,0.021277
518,7,Taiwanese Restaurant,0.021277


In [33]:
toronto_inspect = toronto_inspect.groupby('Cluster Labels').head(5).reset_index(drop=True).copy()
toronto_inspect

,Cluster Labels,Venue Category,venue_perc
0,0,Coffee Shop,0.387719
1,0,Diner,0.084211
2,0,Beer Bar,0.068421
3,0,Café,0.068421
4,0,Cocktail Bar,0.052632
5,1,Greek Restaurant,0.190476
6,1,Coffee Shop,0.071429
7,1,Italian Restaurant,0.071429
8,1,Furniture / Home Store,0.047619
9,1,Ice Cream Shop,0.047619


Lastly, we concatenate the 5 highest occuring venues in each cluster.  This provides a sense of the types of neighbourhoods each cluster represents.

In [34]:
toronto_inspect.groupby('Cluster Labels')['Venue Category'].apply(','.join).reset_index()

,Cluster Labels,Venue Category
0,0,"Coffee Shop,Diner,Beer Bar,Café,Cocktail Bar"
1,1,"Greek Restaurant,Coffee Shop,Italian Restauran..."
2,2,"Garden,Coffee Shop,Park,Café,Restaurant"
3,3,"Café,Bakery,Bar,Bookstore,Japanese Restaurant"
4,4,"Hotel,Pizza Place,Sandwich Place,Gym,Dance Studio"
5,5,"Park,Bus Line,Swim School"
6,6,"Coffee Shop,Park,Café,Trail,Pub"
7,7,"Bakery,Coffee Shop,Café,Italian Restaurant,Market"
